####  Finding optimal hyperparameters for one of the algorithms (LightGBM in this case)

In [1]:
import numpy as np
import pandas as pd

from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn.svm import SVR
from sklearn.neighbors import KNeighborsRegressor

import seaborn as sns
import xgboost as xgb
import lightgbm as lgb
import catboost as cb

import tensorflow as tf
import keras
from keras import layers

#### Load the data

In [2]:
df = pd.read_csv("housing_before_optimization.csv")
df.head()

,housing_median_age,total_rooms,population,median_income,median_house_value,distance_to_nearest_city,ocean_proximity_<1H OCEAN,ocean_proximity_INLAND,ocean_proximity_ISLAND,ocean_proximity_NEAR BAY
0,41.0,880.0,322.0,8.3252,452600.0,20.33,0,0,0,1
1,21.0,7099.0,2401.0,8.3014,358500.0,19.91,0,0,0,1
2,52.0,1467.0,496.0,7.2574,352100.0,17.84,0,0,0,1
3,52.0,1274.0,558.0,5.6431,341300.0,17.06,0,0,0,1
4,52.0,1627.0,565.0,3.8462,342200.0,17.06,0,0,0,1


####  Same X/y + train/test split

In [3]:
categorical_variables = [ 
    "ocean_proximity_<1H OCEAN",
    "ocean_proximity_INLAND",
    "ocean_proximity_ISLAND",
    "ocean_proximity_NEAR BAY"]

# continuous variables also into a list
continuous_variables = [ 
    "housing_median_age",
    "total_rooms",
    "population",
    "median_income",
    "distance_to_nearest_city",
    "median_house_value"]

# the usual X/y -split
X = df.drop("median_house_value", axis=1)
y = df['median_house_value']

# usual train/test -split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

# NOTE! SCALING => some of the algorithms require this
scaler = StandardScaler()

# create separate versions for the scaled data
# because we need both unscaled and scaled versions later
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

####  In this example, use RandomizedSearchCV to search better hyperparameters for our algorithm -  LightGBM

In [4]:
from sklearn.model_selection import RandomizedSearchCV
import numpy as np
from scipy.stats import randint

# Define the parameter distributions with integer ranges
# Notice how we use NumPy and scipy in order to have ranges of values
# for the RandomizedSearch, which it tries to combine in a random manner 
# => based on luck, you might stumble upon a very good combination of parameters
param_dist = {
    'learning_rate': np.linspace(0.08, 0.135, 30), 
    'num_leaves': randint(25, 51),  
    'max_depth': randint(-1, 30), 
}

# Setup RandomizedSearchCV
# search 300 times (n_iter)
# n_jobs => -1 => use all CPU cores
# cv = cross-validation strategy (higher is often better, but takes more time), typical values 3-5
random_search = RandomizedSearchCV(
    estimator=lgb.LGBMRegressor(), 
    param_distributions=param_dist, 
    n_iter=300,  # Number of random combinations to test
    cv=3,
    n_jobs=-1,
    verbose=1,  
    scoring='neg_mean_squared_error'
)

# Fit the randomized search
random_search.fit(X_train, y_train)

# Get the best parameters
best_params_random = random_search.best_params_
print("\nBest parameters from RandomizedSearchCV:", best_params_random)

Fitting 3 folds for each of 300 candidates, totalling 900 fits


c:\Storage\Studies\Lapland_AMK\6_semester\Advanced_ML\venv12\Lib\site-packages\numpy\ma\core.py:2885: RuntimeWarning: invalid value encountered in cast
  _data = np.array(data, dtype=dtype, copy=copy,


[LightGBM] [Warning] Found whitespace in feature_names, replace with underlines
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.000219 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 1079
[LightGBM] [Info] Number of data points in the train set: 15580, number of used features: 8
[LightGBM] [Info] Start training from score 192477.721181

Best parameters from RandomizedSearchCV: {'learning_rate': np.float64(0.13120689655172416), 'max_depth': 23, 'num_leaves': 46}


In [5]:
# Taken from AI to understand if the model is overfitting or not. According to AI if train > test → overfitting and i need to reduce max_depth
# If train and test are about the same than i am good. 
# In my case -df without dropping rows that are higher than 500 001 -  train_score-1900641404.2566435, test_score-2604742485.6073427) i am good with these parameters, because test is not not wildly higher than train.
# PS1 - using df before optimization, i got train_score = -1968491890.4575675 and test_score = -2972746424.5103645. According to Ai, 
# I need to  change max_depth from 'max_depth': randint(-1, 40) to "max_depth": randint(4, 10) for example.
# PS2 - df before optimization, train_score = -2191108649.1476135 and test_score = -3034239139.642399. A bit better but still quite a big gap.

train_score = random_search.score(X_train, y_train)
test_score  = random_search.score(X_test, y_test)

print(train_score, test_score)

-1594676715.389762 -2688690636.762296
